In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
import shutil

In [89]:
# Define Paths
data_dir = "C:\\Users\\mvish\\Projects\\cheatshield\\data\\face_images\\widerface"
train_dir = "./data/wider_face_train"
val_dir = "./data/wider_face_val"
IMG_HEIGHT=128
IMG_WIDTH=128
BATCH_SIZE = 32
EPOCHS = 20

In [90]:
# Ensure train and val directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [91]:
# Manually split dataset if not already done
if not os.listdir(train_dir) or not os.listdir(val_dir):
    images = os.listdir(data_dir)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Move images to train and val directories
    for img in train_images:
        shutil.copy(os.path.join(data_dir, img), os.path.join(train_dir, img))
    for img in val_images:
        shutil.copy(os.path.join(data_dir, img), os.path.join(val_dir, img))

In [92]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,                     # Normalize pixel values to [0, 1]
    rotation_range=10,                  # Randomly rotate images by 10 degrees
    width_shift_range=0.1,              # Randomly shift images horizontally by 10%
    height_shift_range=0.1,             # Randomly shift images vertically by 10%
    horizontal_flip=True                # Randomly flip images horizontally
)

In [93]:
# Load Data
train_data = datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
val_data = datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

Found 2558 images belonging to 61 classes.
Found 668 images belonging to 61 classes.


In [94]:
# Load Pretrained ResNet50 Model with proper input shape
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in base_model.layers[:-50]:
    layer.trainable = False  # Freeze initial layers

# Add Custom Layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(1, activation='sigmoid')(x)  # Binary Classification

model = Model(inputs=base_model.input, outputs=out)

TypeError: list indices must be integers or slices, not str